In [1]:
!pip install openai

In [2]:
from openai import OpenAI
import subprocess

In [3]:
groq = OpenAI(api_key = "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", base_url = "https://api.groq.com/openai/v1")

In [4]:
def reencoded_audio_to_ogg(input_file, output_file = "encoded_audio.ogg"):
  command = [
    "ffmpeg", "-y", "-i", input_file,     # Input file
    "-vn",                                # Remove any video stream (if present)
    "-map_metadata", "-1",                # Strip all metadata (title, artist, etc.)
    "-ac", "1",                           # Set audio to mono (1 channel)
    "-c:a", "libopus",                    # Use the Opus codec (great for speech)
    "-b:a", "12k",                        # Set a very low bitrate (12 kbps)
    "-application", "voip",               # Tune Opus codec for voice (VoIP mode)
    output_file                           # Output file name
  ]
  subprocess.run(command, check=True)

In [5]:
def transcibe_audio(file_path):
  print(f"[INFO] Re-encoding {file_path} to OGG Format....")
  reencoded_audio_to_ogg(file_path, "encoded_audio.ogg")
  print(f"[INFO] Transcribing With Groq Whisper")
  with open("/content/encoded_audio.ogg", "rb") as f:
    transcript = groq.audio.transcriptions.create(
        model = "whisper-large-v3",
        file = f,
        response_format="text"
    )

  return transcript


In [6]:
from google.colab import files
uploaded = files.upload()
filename = next(iter(uploaded))

transcription = transcibe_audio(filename)
print("\nTranscription Result: \n")
print(transcription)

StopIteration: 

In [7]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.6 MB/s eta 0:00:00


In [8]:
import gradio as gr
app = gr.Interface(
    fn = transcibe_audio,
    inputs = gr.Audio(type = "filepath", label = "Upload Audio"),
    outputs = gr.Textbox(label = "Transcription"),
    title="🎤 Groq Whisper Transcriber",
    description="Upload an audio file, and Groq Whisper will transcribe it using the large-v3 model.",
    theme="soft"
)

In [9]:
if __name__ == "__main__":
  app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93b30166cc3105f0a3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
